# Reformat the data from Ridley+2023 on AT2017bcc into the OTTER JSON format

### Metadata First

In [1]:
import os
import pandas as pd
import numpy as np
import otter

from astropy import units as u

from dataclasses import dataclass

@dataclass
class Args:
    otterdir: str
    indir: str
    
@dataclass
class Param:
    name: float | str | int
    unit: str
    description: str = None

    
args = Args(
    otterdir=".otter",
    indir=os.getcwd()
)

month_map = dict(
    Jan = "01",
    Feb = "02",
    Mar = "03",
    Apr = "04",
    May = "05",
    Jun = "06",
    Jul = "07",
    Aug = "08",
    Sep = "09",
    Oct = "10",
    Nov = "11",
    Dec = "12"
)


In [2]:
hajela24_bibcode = "2024arXiv240719019H"

meta = dict(
    name = dict(
        default_name = "ASASSN-15oi",
        alias = [dict(value="ASASSN-15oi", reference=[hajela24_bibcode])]
    ),

    coordinate = [dict(
        reference = ["2016MNRAS.463.3813H"],
        ra = "20:39:09.12",
        dec = "-30:45:20.84",
        ra_units = "hourangle",
        dec_units = "deg",
        default = True,
        coordinate_type = "equitorial"
    )],
    
    reference_alias = [
        dict(name=hajela24_bibcode, human_readable_name="Hajela et al. (2024)"),
        dict(name="2016MNRAS.463.3813H", human_readable_name="Holoien et al. (2016)"),
        dict(name="2020PASP..132c5001L", human_readable_name="Lucy et al. (2020)"),
        dict(name="2017ApJ...851L..47G", human_readable_name="Gezari et al. (2017)")
    ]
)

meta

{'name': {'default_name': 'ASASSN-15oi',
  'alias': [{'value': 'ASASSN-15oi', 'reference': ['2024arXiv240719019H']}]},
 'coordinate': [{'reference': ['2016MNRAS.463.3813H'],
   'ra': '20:39:09.12',
   'dec': '-30:45:20.84',
   'ra_units': 'hourangle',
   'dec_units': 'deg',
   'default': True,
   'coordinate_type': 'equitorial'}],
 'reference_alias': [{'name': '2024arXiv240719019H',
   'human_readable_name': 'Hajela et al. (2024)'},
  {'name': '2016MNRAS.463.3813H',
   'human_readable_name': 'Holoien et al. (2016)'},
  {'name': '2020PASP..132c5001L', 'human_readable_name': 'Lucy et al. (2020)'},
  {'name': '2017ApJ...851L..47G',
   'human_readable_name': 'Gezari et al. (2017)'}]}

### UVOIR Dataset

In [3]:
uvoir = pd.read_csv(os.path.join(args.indir, "asassn15oi_uvoir.txt"), sep=" ", index_col=False)

uvoir_phot = dict(
    reference = [hajela24_bibcode],
    raw = uvoir.Magnitude.tolist(),
    raw_err = uvoir.Magnitude_Error.tolist(),
    raw_units = "mag(AB)",
    corr_k = False,
    corr_s = False,
    corr_av = True,
    corr_host = False,
    corr_hostav = False,
    val_av = 0.185,
    upperlimit = [False]*len(uvoir),
    date = uvoir.MJD.tolist(),
    date_format = "MJD",
    filter_key = uvoir["Filter"].tolist(),
    obs_type = "uvoir"
)

filts, idxs = np.unique(uvoir["Filter"], return_index=True)
print(filts)
filter_alias = [
    dict(
        filter_key = f,
        filter_name = f,
        wave_eff = otter.util.FILTER_MAP_WAVE[f],
        wave_units = "nm"
    ) for f in filts
]

# test the otter dataset format based on our schema
otter.schema.PhotometrySchema(**uvoir_phot);
for f in filter_alias:
    otter.schema.FilterSchema(**f);

['b' 'u' 'uvm2' 'uvw1' 'uvw2' 'v']


### Radio Dataset

In [4]:
radio = pd.read_csv("asassn15oi_radio.txt", sep=" ")
radio_srcmap = {
    "this": [hajela24_bibcode],
    "VLASS": [hajela24_bibcode, "2020PASP..132c5001L"]
}

radio["iso"] = radio.Date.replace(month_map, regex=True)

radio_phot = [
    dict(
        reference = radio_srcmap[src],
        raw = grp.Fν_mJy.tolist(),
        raw_err = grp.Fv_err.tolist(),
        raw_units = "mJy",
        corr_k = False,
        corr_s = False,
        corr_av = False,
        corr_host = False,
        corr_hostav = False,
        upperlimit = grp.upperlimit.tolist(),
        date = grp.iso,
        date_format = "iso",
        filter_key = (grp.ν_GHz.astype(str)+["GHz"]*len(grp)).tolist(),
        obs_type = "radio",
        telescope = tele
    ) for (tele, src), grp in radio.groupby(["Observatory", "Source"])
]

filter_alias += [
    dict(
        filter_key = str(f)+"GHz",
        filter_name = otter.util.freq_to_band(f*u.GHz),
        freq_eff = f,
        freq_units = "GHz"
    ) for f in radio.ν_GHz.unique()
]

# test the otter dataset format based on our schema
for r in radio_phot:
    otter.schema.PhotometrySchema(**r);
for f in filter_alias:
    otter.schema.FilterSchema(**f);

/home/nfranz/.local/lib/anaconda3/lib/python3.11/site-packages/pydantic/main.py:193: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  self.__pydantic_validator__.validate_python(data, self_instance=self)


### X-ray dataset

In [5]:
xray = pd.read_csv(os.path.join(args.indir, "asassn15oi_xray.txt"), sep=" ")

xray_srcmap = {
    "this" : [hajela24_bibcode],
    "Gezari17" : [hajela24_bibcode, "2017ApJ...851L..47G"],
    "Holoien18": [hajela24_bibcode, "2016MNRAS.463.3813H"]
}

band_map = {
    "swift" : "0.3 - 10",
    "XMM-Newton" : "0.2 - 12"
}

xray["model_name"] = [
    "Absorbed Powerlaw + Blackbody",
    "Absorbed Powerlaw + Blackbody",
    "Absorbed Powerlaw + Blackbody",
    "Absorbed Powerlaw + Blackbody",
    "Absorbed Powerlaw + Blackbody",
    "Absorbed Powerlaw",
    "Absorbed Powerlaw",
    "Powerlaw + Blackbody",
    "Absorbed Powerlaw + Blackbody",
    "Absorbed Powerlaw + Blackbody",
    "Powerlaw + Blackbody",
    "Absorbed Powerlaw + Blackbody",
    "Absorbed Powerlaw + Blackbody",
    "Absorbed Powerlaw + Blackbody",
    "Absorbed Powerlaw"
]

gPL = Param(name="gamma_PL", unit="None", description="Photon Index for Powerlaw model")
FPL = Param(name="F_PL", unit="10^-14 erg/cm^2/s", description="Unabsorbed flux for Powerlaw model")
kT = Param(name="kT_BB", unit="10^-2 keV", description="k*T for the Blackbody model")
FBB = Param(name="F_BB", unit="10^-13 erg/cm^2/s", description="Unabsorbed flux for Blackbody model")
RBB = Param(name="R_BB", unit="10^12 cm", description="Blackbody radius")
all_ = [gPL, FPL, kT, FBB, RBB]
xray["param_names"] = [
    all_,
    all_,
    all_,
    all_,
    [gPL,  FPL, kT, FBB],
    [gPL, FPL],
    [gPL, FPL],
    [gPL, kT, FBB, RBB],
    all_,
    all_,
    [gPL, kT, FBB, RBB],
    all_,
    all_,
    [gPL,  FPL, kT, FBB],
    [gPL, FPL]
]

for p in all_:
    xray[[p.name, p.name+"_lower", p.name+"_upper"]].astype(float)

def compute_flux(row):
    if not pd.isna(row.F_PL) and not pd.isna(row.F_BB) and not row.F_PL_upperlimit and not row.F_BB_upperlimit:
        return (
            row.F_PL*1e-14 + row.F_BB*1e-13, 
            (
                np.sqrt((row.F_PL_upper*1e-14)**2 + (row.F_BB_upper*1e-13)**2) + 
                np.sqrt((row.F_PL_lower*1e-14)**2 + (row.F_BB_lower*1e-13)**2)
            )/2,
            False,
            np.sqrt((row.F_PL_upper*1e-14)**2 + (row.F_BB_upper*1e-13)**2),
            np.sqrt((row.F_PL_lower*1e-14)**2 + (row.F_BB_lower*1e-13)**2)
        )
            
    elif (pd.isna(row.F_PL) or row.F_PL_upperlimit) and not pd.isna(row.F_BB) and not row.F_BB_upperlimit:
        return row.F_BB*1e-13, (row.F_BB_upper + abs(row.F_BB_lower))/2 * 1e-13, False, row.F_BB_upper, abs(row.F_BB_lower)
    
    elif (pd.isna(row.F_BB) or row.F_BB_upperlimit) and not pd.isna(row.F_PL) and not row.F_PL_upperlimit:
        return row.F_PL*1e-14, (row.F_PL_upper + abs(row.F_PL_lower))/2 * 1e-14, False, row.F_PL_upper, abs(row.F_PL_lower)
    
    elif row.F_PL_upperlimit:
        return row.F_PL*1e-14, 0, True, 0, 0
    
    else:
        print(row)
        raise ValueError()

xray["flux"], xray["flux_err"], xray["upperlimit"], xray["flux_upper"], xray["flux_lower"] = list(zip(*xray.apply(compute_flux, axis=1).tolist()))
        
N_H = 5.6e20 # cm^-2

xray_phot = [
    dict(
        reference = xray_srcmap[src],
        raw = grp.flux.tolist(),
        raw_err = grp.flux_err.tolist(),
        raw_units = "erg/s/cm^2",
        corr_k = False,
        corr_s = False,
        corr_av = False,
        corr_host = False,
        corr_hostav = False,
        val_host = (grp.F_abs*1e-14).tolist(),
        upperlimit = grp.upperlimit.tolist(),
        date = grp.dt + 57248.0,
        date_format = "MJD",
        filter_key = band_map[tele],
        obs_type = "xray",
        telescope = tele,
        raw_err_detail = dict(
            upper = xray.flux_upper.tolist(),
            lower = xray.flux_lower.tolist()
        ),
        xray_model = [
            dict(
                model_name = row.model_name,
                param_names = [p.name for p in row.param_names],
                param_values = [row[name.name] for name in row.param_names],
                param_units = [p.unit for p in row.param_names],
                param_value_err_upper = [row[name.name+"_upper"] for name in row.param_names],
                param_value_err_lower = [row[name.name+"_lower"] for name in row.param_names],
                param_upperlimit = [row[name.name+"_upperlimit"] for name in row.param_names],
                param_descriptions = [p.description for p in row.param_names],
                model_reference = [hajela24_bibcode],
                min_energy = float(band_map[tele].split('-')[0].strip()),
                max_energy = float(band_map[tele].split()[-1].strip()),
                energy_units = "keV"
            ) for _, row in grp.iterrows()
        ]
    ) for (src, tele), grp in xray.groupby(["source", "telescope"])
]

filter_alias += [
    dict(
        filter_key = "0.3 - 10",
        filter_name = "0.3 - 10",
        wave_eff = ((10-0.3)*u.keV).to(u.nm, equivalencies=u.spectral()).value,
        wave_units = "nm",
        wave_max = (0.3*u.keV).to(u.nm, equivalencies=u.spectral()).value,
        wave_min = (10*u.keV).to(u.nm, equivalencies=u.spectral()).value
    ),
    dict(
        filter_key = "0.2 - 12",
        filter_name = "0.2 - 12",
        wave_eff = ((12-0.2)*u.keV).to(u.nm, equivalencies=u.spectral()).value,
        wave_units = "nm",
        wave_max = (0.2*u.keV).to(u.nm, equivalencies=u.spectral()).value,
        wave_min = (12*u.keV).to(u.nm, equivalencies=u.spectral()).value
    ),
]

# test the otter dataset format based on our schema
for r in xray_phot:
    otter.schema.PhotometrySchema(**r);
for f in filter_alias:
    otter.schema.FilterSchema(**f);

# Merge everything

In [6]:
otter_json = meta | {
    "photometry" : [uvoir_phot]+radio_phot+xray_phot,
    "filter_alias" : filter_alias
}

otter.schema.OtterSchema(**otter_json)

otter.Transient(otter_json)

/home/nfranz/.local/lib/anaconda3/lib/python3.11/site-packages/pydantic/main.py:193: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  self.__pydantic_validator__.validate_python(data, self_instance=self)


Transient(
	Name: ASASSN-15oi,
	Keys: dict_keys(['name', 'coordinate', 'reference_alias', 'photometry', 'filter_alias'])
)

In [7]:
db = otter.Otter(datadir=args.otterdir)

db.save([otter.Transient(otter_json)], testing=True)

ASASSN-15oi
Adding this as a new object...
Don't need to mess with the files at all!
Would write to .otter/ASASSN-15oi.json
